# Replica project

### The data

In [63]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import Image as Image2
import numpy as np

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

from urllib.request import urlretrieve 
import pickle
from glob import glob
from scipy import sparse

from torch.utils.data import DataLoader

from utils import *
from dataloader_replica import ReplicaDataset
from train_replica import train_replica
from model_replica import ReplicaNet

import umap

from utils import * 
path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata.head(5)

/tmp/ipykernel_1529928/2286784503.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorDeath,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,1690.0,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,1612.0,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,1622.0,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,1971.0,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...


In [ ]:
metadata['uid'].nunique(), metadata.shape

(329480, (330002, 37))

In [ ]:
# dropping duplicates

metadata = metadata.groupby('uid').first().reset_index()

In [ ]:
# in jeanne's code
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

,uid,img1,img2,type,annotated
0,0f84770be893475581bebd8c0ce95704,4b6e5a18012e4974a43f6f652c9cad78,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:50.997512+00:00
1,5ade49fc2dc649b9947496a41e2decdb,384b9f017d5f43ba9dcfd92f2af830c4,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:51.238615+00:00
2,bf4d99eaf7da4bd486f9007c90c177a5,94fe74bcebf9442fb5cd9a0327590a84,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:23.705228+00:00
3,b29ec3c83c5641f9b6afadb2409b6746,922fe92d61e744b1a9462c848a916bcf,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:24.021703+00:00
4,ab27b55997b94bef9cca122829e9f64d,94fe74bcebf9442fb5cd9a0327590a84,922fe92d61e744b1a9462c848a916bcf,POSITIVE,2017-07-20 12:18:23.873097+00:00


In [ ]:
morpho_graph_complete['type'].value_counts() # does positive mean similar?

DUPLICATE        114283
POSITIVE           6293
NON-DUPLICATE        60
PROPOSAL             27
Name: type, dtype: int64

In [ ]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120663, (120663, 5))

In [ ]:
positives = get_train_test_split(metadata, morpho_graph_complete)

In [ ]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)
positives.head()

(2862, 45)


,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
0,002ca9c06a704c779e69d93cb4fadaa3,649,45588b4e33444ed8b34c523bac808bc4,158B,556,TIZIANO Vecellio (copia da),S.Mariá Maddalena. cm. 123 x 91,TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158B%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50,train
1,0038ab462149452b98a4d7359a06f9f4,1571,ae828b07e4284d80b652206ffb77278a,158C,120,TIZIANO Vecellio (copia da),"Morte di S. Pietro Martire. mm. 123,8 x 84,2",TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158C%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51,test
2,00474897de044f7fb1ef16888dc198f3,1107,76b2c4aeab404bc79768117e3ddc62b1,47A,858,BELLINI Giovanni (cerchia di),Pietà.,"BELLINI, Giovanni (cerchia di -)","Bellini, Giovanni",ulan:500019244,...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/47A%2F47A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498,test
3,009c03c265b8418495da038bc3826bf7,1764,c5839e5220e5419e92d878396b0c04bb,135C,57,SEC XVI ex,Un soffitto. Part.: Leda e il cigno.,SEC. XVI ex.,None,None,...,None,None,https://dhlabsrv4.epfl.ch/iiif_cini/135C%2F135...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,450ee2aceb014709b23a93e1163937c5,c5839e5220e5419e92d878396b0c04bb,POSITIVE,2018-05-07 08:28:11.797723+00:00,92,train
4,00aeddb93a8a4120aa984eda6c822747,747,4d02860f58f2443ba4d1ef088756f854,21C,228,BASSANO Leandro,"La preparazione del pranzo. cm. 88,3 x 102,9","BASSANO, Leandro.","Bassano, Leandro",ulan:500015945,...,Bassano del Grappa,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,4d02860f58f2443ba4d1ef088756f854,27bad0f213a342c3987e8cec61e40550,POSITIVE,2018-03-23 15:41:53.094226+00:00,218,train


## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [ ]:
subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()
subset_max.shape

/tmp/ipykernel_1529928/3952239037.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()


(3302, 38)

In [ ]:
list_downloaded = [file.split('/')[-1].split('.')[0] for file in glob(data_dir + 'subset/*')]
list_downloaded[0]

'383d8fb371ae4aa9bd3819fb973f0ac9'

In [ ]:
len(list_downloaded)

4593

In [ ]:
del morpho_graph_complete
del metadata

In [ ]:
uniques = subset_max[subset_max['uid'].isin(list_downloaded)].groupby('uid').first().reset_index()
print(uniques.shape)

(3301, 38)


In [16]:
data = positives.merge(uniques, left_on='uid', right_on='uid', how='outer')
print(data.shape)

data.to_csv(data_dir + 'full_data.csv')

(4357, 45)


In [22]:
data.sample(5)

,uid,index,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,...,AuthorDeathCity,CardboardURL,ImageURL,uid_connection,img1,img2,type,annotated,cluster,set
1015,3ac12680fbfe41d98e168a9b3f692dc5,76156,33B,418,SPADA Lionello,Studi. mm. 272 x 196,"SPADA, Iionello.","Spada, Leonello",ulan:500030547,http://vocab.getty.edu/page/ulan/500030547,...,Parma,https://dhlabsrv4.epfl.ch/iiif_cini/33B%2F33B_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,None,NaT,NaN,None
4292,fc907a24254e4c78a1d0f0b52d877251,2268,28A,195,RAFAELLO (copia da),"Madonna col Putto. Sanguigna, mm. 270 x 203.",RAFF AELLO. (copia da -),Raphael,ulan:500023578,http://vocab.getty.edu/page/ulan/500023578,...,Rome,https://dhlabsrv4.epfl.ch/iiif_cini/28A%2F28A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ee90d63e0eac48409aebea03d996897a,fc907a24254e4c78a1d0f0b52d877251,04d6cf0ce2974c7fa35552f2a0ec67db,POSITIVE,2018-07-14 15:01:33.163979+00:00,843.0,test
3305,c11a5578c53f4e60b89b9bda1326d2fc,248394,25C,233,RENI Guido,La Sacra Famiglia. 366 x 277.,"RENI, Guido.","Reni, Guido",ulan:500030334,http://vocab.getty.edu/page/ulan/500030334,...,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/25C%2F25C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,None,NaT,NaN,None
2624,9a0c1575b55b4703b413bca06bfeb4aa,198142,114C,167,RENI Guido,Gesù Crocifisso.,"RENI, Guido.","Reni, Guido",ulan:500030334,http://vocab.getty.edu/page/ulan/500030334,...,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/114C%2F114...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,None,NaT,NaN,None
255,0ed1d066fd2045668094ea21cd54659b,19161,136C,296,GUERCINO (BARBIERI Giovanni Francesco),Sacra Famiglia.,GUERCINO (IL),Guercino,ulan:500021925,http://vocab.getty.edu/page/ulan/500021925,...,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/136C%2F136...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,None,NaT,NaN,None


### Creating embeddings of all the images

In [36]:
model = models.resnet50(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [40]:
newmodel = torch.nn.Sequential(
    *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1,1))
)
print(newmodel)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [57]:
img = preprocess_image(data_dir + 'subset/' + data['uid'][0] + '.jpg')
embedding = get_embedding(img, newmodel).squeeze(3).squeeze(1).squeeze(0)

In [61]:
embeddings = [get_embedding(preprocess_image(data_dir + 'subset/' + uid + '.jpg'), newmodel).squeeze(3).squeeze(1).squeeze(0) for uid in tqdm(data['uid'])]

100%|██████████| 4357/4357 [17:53<00:00,  4.06it/s]


In [67]:
embeddings_new = get_lower_dimension(embeddings)

In [68]:
embeddings[0].shape, embeddings_new[0].shape 

((2048,), (100,))

### Finding most similar embeddings

In [69]:
tree = make_tree_list(embeddings_new)

In [77]:
train_test = data[data['set'].notnull()].reset_index()

list_theo = list(train_test[train_test['img1'] == train_test['uid'][0]]['img2']) + list(train_test[train_test['img2'] == train_test['uid'][0]]['img1']) + [train_test['uid'][0]]
print(list_theo)

list_ = find_most_similar_list(train_test['uid'][0], tree, embeddings_new, list(data['uid']), list_theo)
print(list_)

['a15f8efc3926415da7727b63ab819ee5', '00a59b7eb3ae41af81d3ccaaedb2007c']
['5bf38032297649ada8220c4089cbcd1b', '6a6be020a2314518baae3d83994b6efd', '71c79a728c7445449dfa30e5eda3b5d9', '29ef552c972f487cb17b4c316bc3df52', '7560c69edcac41438083be487fc44d21', 'ba5520b7d7f54712a3431b1c2bd37745', '357e633ae3534938b9cb1c395f9e4de9', 'bf5fcf18d4834a0095d0db31912e2679', '2103e56b57504b0e9973c783e3160c3c', 'a830e6b7dd2445e0a2f88c892c28f613', 'bfdebcc918a244b5b6c08334f6b3752e', '97be8d9f056b46ce8c800abcd6df4985', '2869a411e80e417885cceb1a2a83ead5', '8ec5e70f631d47af837b643e42df9680', 'e91819b521274ae88278116f0c86aedb', '4fc46726c0da4483bb0e23da47245839', 'd03d35e53e444d95b0e1c44f06b74880', 'e471e13cb38a460fbbd28d4fe66b3d14', 'a6b59b98c2874a14933c8d6f605c10d1']
